In [2]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import mysql.connector

# MySQL 연결 함수
def get_db_connection():
    return mysql.connector.connect(
        host="localhost",
        user="ohgiraffers",
        password="ohgiraffers",
        database="test_db"
    )

# 사용자 느낌에 맞는 향수 데이터를 DB에서 가져오는 함수
def get_fragrance_from_db(user_feeling):
    conn = get_db_connection()
    cursor = conn.cursor(dictionary=True)
    
    # 느낌에 해당하는 향수 데이터를 찾기 위한 쿼리
    query = """
    SELECT name, brand, description
    FROM perfume
    WHERE description LIKE %s
    """
    
    cursor.execute(query, ('%' + user_feeling + '%',))
    result = cursor.fetchall()
    cursor.close()
    conn.close()
    
    return result

# GPT-4에게 추천 이유를 작성하게 하는 함수
def generate_recommendation_reason(user_feeling, perfume=None):
    if perfume:
        # DB에서 추천할 향수 정보를 포함하여 프롬프트 생성
        perfume_info = "\n".join(
            [f"이름: {f['name']}, 브랜드: {f['brand']}, 설명: {f['description']}" for f in perfume]
        )
        prompt_template = """
        사용자가 '{user_feeling}'라는 느낌을 요청했습니다.
        참고할 수 있는 향수 목록은 다음과 같습니다:
        {perfume}
        
        사용자에게 가장 적합한 향수를 추천하고, 이유를 작성해 주세요.
        """
        prompt = PromptTemplate(input_variables=["user_feeling", "perfume"], template=prompt_template)
    else:
        # DB에 적합한 향수가 없을 때 일반적인 추천을 위한 프롬프트 생성
        prompt_template = """
        사용자가 '{user_feeling}'라는 느낌을 요청했습니다.
        이 느낌에 어울리는 향수를 추천하고, 추천 이유를 작성해 주세요.
        """
        prompt = PromptTemplate(input_variables=["user_feeling"], template=prompt_template)

    # LangChain을 사용하여 LLMChain 생성 및 GPT-4 요청
    chain = prompt | ChatOpenAI(model="gpt-4")
    input_data = {"user_feeling": user_feeling, "perfume": perfume_info} if perfume else {"user_feeling": user_feeling}
    response = chain.invoke(input_data)

    return response['text'] if 'text' in response else response

# 사용자 요청에 따른 향수 추천을 위한 함수
def recommend_fragrance_based_on_feeling(user_feeling):
    # DB에서 향수 데이터 가져오기
    perfume = get_fragrance_from_db(user_feeling)
    
    # DB에서 추천할 향수가 있을 경우 이를 사용하여 추천 이유 생성
    if perfume:
        recommendation = generate_recommendation_reason(user_feeling, perfume)
    else:
        # DB에 적합한 향수가 없을 경우 외부 추천 요청
        recommendation = generate_recommendation_reason(user_feeling)
    
    return {
        "recommendation": recommendation,
        "perfume_list": perfume if perfume else "외부 추천을 통해 제공된 향수"
    }

# 사용자 요청 처리 예시
user_feeling = "소중한 날을 위한 특별한 향수를 추천해 주세요."
response = recommend_fragrance_based_on_feeling(user_feeling)

# 결과 출력
print("[느낌]:\n" + "-"*40)
print(user_feeling)
print("\n[추천 이유]:\n" + "-"*40)
print(response["recommendation"])

print("\n[추천 향수]:\n" + "-"*40)
if isinstance(response["perfume_list"], str):
    print(response["perfume_list"])
else:
    for perfume in response["perfume_list"]:
        print(f"이름: {perfume['name']}, 브랜드: {perfume['brand']}, 설명: {perfume['description']}")
        print("-" * 40)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable